In [ ]:
!pip install boto3 -q

In [ ]:
import json
import boto3
from IPython.display import clear_output, display, display_markdown, Markdown

## Create the boto3 client

Interaction with the Bedrock API is done via the AWS SDK for Python: [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html).

Depending on your environment, you might need to customize the setup when creating your Bedrock service client.

In [ ]:
bedrock = boto3.client(
 service_name='bedrock',
 region_name='us-east-1'
)

bedrock_runtime = boto3.client(
 service_name='bedrock-runtime',
 region_name='us-east-1',
)

In [ ]:
# list available models
bedrock.list_foundation_models()

## `InvokeModel` body and output

The `invoke_model()` method of the Amazon Bedrock client (`InvokeModel` API) will be the primary method we use for most of our Text Generation and Processing tasks - whichever model we're using.

Although the method is shared, the format of input and output varies depending on the foundation model used - as described below:

### Amazon Titan Large

#### Input
```json
{   
    "inputText": "<prompt>",
    "textGenerationConfig" : { 
        "maxTokenCount": 512,
        "stopSequences": [],
        "temperature": 0.1,  
        "topP": 0.9
    }
}
```

#### Output

```json
{
    "inputTextTokenCount": 613,
    "results": [{
        "tokenCount": 219,
        "outputText": "<output>"
    }]
}
```

### Anthropic Claude

#### Input

```json
{
    "prompt": "\n\nHuman:<prompt>\n\nAnswer:",
    "max_tokens_to_sample": 300,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman:"]
}
```

#### Output

```json
{
    "completion": "<output>",
    "stop_reason": "stop_sequence"
}
```

### Stability AI Stable Diffusion XL

#### Input

```json
{
    "text_prompts": [
        {"text": "this is where you place your input text"}
    ],
    "cfg_scale": 10,
    "seed": 0,
    "steps": 50
}
```

#### Output

```json
{ 
    "result": "success", 
    "artifacts": [
        {
            "seed": 123, 
            "base64": "<image in base64>",
            "finishReason": "SUCCESS"
        },
        //...
    ]
}
```

### Simple prompt

In [ ]:
prompt_data = "Human: Escreva um artigo sobre o Brasil Assistant:"

In [ ]:
body = json.dumps({
    "prompt": prompt_data,
    "max_tokens_to_sample": 3072,
    "stop_sequences": ["\n\nHuman:"],
    "temperature": 0.1,  
    "top_p": 0.9,
    "top_k": 256
})
modelId = "anthropic.claude-v2"
accept = "application/json"
contentType = "application/json"

response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("completion"))

### Zero-shot

Zero Shot prompting describes the technique where we present a task to an LLM without giving it further examples. We therefore, expect it to perform the task without getting a prior look at a “shot” at the task. Hence, “zero-shot” prompting. Modern LLMs demonstrate remarkable zero-shot performance and a positive correlation can be drawn between model size and zero-shot performance.

In [ ]:
prompt_data = """Human: Sulfuric acid reacts with sodium chloride, and gives <chemical1>_____</chemical1> and <chemical2>_____</chemical2>:
Assistant: the chemical1 and chemical 2 are: Assistant:"""

In [ ]:
body = json.dumps({
    "prompt": prompt_data,
    "max_tokens_to_sample": 3072,
    "stop_sequences": ["\n\nHuman:"],
    "temperature": 0.1,  
    "top_p": 0.9,
    "top_k": 256
})
modelId = "anthropic.claude-v2"
accept = "application/json"
contentType = "application/json"

response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("completion"))

### Few-shot

Giving the model more information about the tasks at hand via examples is called Few-Shot Prompting. It can be used for in-context learning by providing examples of the task and the desired output. We therefore condition the model on with the examples to follow the task guidance more closely.

In [ ]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """
Human:
GME to the moon // Neutral
APPL is going down // Negative
BA is moving sideways since forever // Negative
Bearish on SPY. The economy is going down // Positive
AMZN is going through the roof! //

Assistant:
"""

In [ ]:
body = json.dumps({
    "prompt": prompt_data,
    "max_tokens_to_sample": 3072,
    "stop_sequences": ["\n\nHuman:"],
    "temperature": 0.1,  
    "top_p": 0.9,
    "top_k": 256
})
modelId = "anthropic.claude-v2"
accept = "application/json"
contentType = "application/json"

response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("completion"))

### Chain-of-Thought (with Few-Shot)
Chain-of-Thoughts (CoT) prompting breaks down complex reasoning tasks through intermediary reasoning steps. Chain-of-Thought prompts usually are very specific to a problem type. One can try to invoke CoT reasoning by using the trigger phrase “(Think Step-by-Step)”. Lets examine the below example of such a few-shot CoT prompt.

In [ ]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """
Human:

On a given week, the viewers for a TV channel were
Monday: 6500 viewers
Tuesday: 6400 viewers
Wednesday: 6300 viewers


Question: How many viewers can we expect on Friday?
Answer: Based on the numbers given and without any more information, there is a daily decrease of 100 viewers. If we assume this trend will continue during the following days, we can expect 6200 viewers on the next day that would be Thursday, and therefore 6100 viewers on the next day that would be Friday.


Question: How many viewers can we expect on Saturday? (Think Step-by-Step)
Answer:

Assistant:

"""

In [ ]:
body = json.dumps({
    "prompt": prompt_data,
    "max_tokens_to_sample": 3072,
    "stop_sequences": ["\n\nHuman:"],
    "temperature": 0.1,  
    "top_p": 0.9,
    "top_k": 256
})
modelId = "anthropic.claude-v2"
accept = "application/json"
contentType = "application/json"

response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("completion"))